*This notebook is devoted to creating and training the autoencoder which will later be used to generate the art*

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Input
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dense
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
import shutil
import os
from os import listdir
import numpy as np

*My images begin as 224x224 pixel images with 3 layers for RGB. Through convolutions and maxpooling, they are condensed into an encoded tensor of dimensions 28x28x16. Hense an initial image of 150,528 features is shrunk to a size of 12,544 features, less than a tenth the original size, before then being decoded to recreate the original image of the same size.*

In [115]:
input_ = Input(shape=(224,224,3))

x = Conv2D(512, (3,3), activation = 'relu', padding='same')(input_)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(64, (3,3), activation = 'relu', padding='same')(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(16, (3,3), activation = 'relu', padding='same')(x)
encoded = MaxPooling2D((2,2), padding='same', name='encoded')(x)

x = Conv2D(16, (3,3), activation = 'relu', padding='same')(encoded)
x = UpSampling2D((2,2))(x)
x = Conv2D(64, (3,3), activation = 'relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
x = Conv2D(512, (3,3), activation = 'relu', padding='same')(x)
x = UpSampling2D((2,2))(x)
decoded = Conv2D(3, (3,3), activation = 'sigmoid', padding='same')(x)


autoencoder = Model(input_, decoded)
autoencoder.compile(optimizer='adam', loss='mse', metrics=['acc'])

In [116]:
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 224, 224, 512)     14336     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 512)     0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 112, 112, 64)      294976    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 56, 56, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 56, 56, 16)        9232      
_________________________________________________________________
encoded (MaxPooling2D)       (None, 28, 28, 16)        0   

*In order to create both a testing and validation directory, I move 2,500 images randomly from the original directory to a validation directory*

In [24]:
images = listdir('../images/neural_image_bucket/ads/more_pinterest')
valid_ind = np.random.choice(len(images), 2500, replace=False)
for ind in valid_ind:
    shutil.move('../images/neural_image_bucket/ads/more_pinterest/'+images[ind], 
                    '../images/neural_image_bucket/validation/images/'+images[ind])

*I then create paths to each directory and create generators to feed images to the autoencoder while training*

In [72]:
training_directory = os.path.join('../images/neural_image_bucket/ads/')
validation_directory = os.path.join('../images/neural_image_bucket/validation/')

datagen = ImageDataGenerator(rescale=1./255) #this normalizes the input as all pixel strength is out of 255
train = datagen.flow_from_directory(
                training_directory,
                target_size=(224, 224), #input size for the generator
                batch_size=20, #limit the batch size so as not to overwork the computer
                class_mode='input') #the final output should be the same as the input

validate = datagen.flow_from_directory(
                validation_directory,
                target_size=(224, 224),
                batch_size=20,
                class_mode='input')

Found 10087 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [119]:
#Model Checkpoint automatically saves the best performing model to date for optimization in training
checkpoint = ModelCheckpoint('../auto_saving.h5', verbose=1, save_best_only=True)

In [75]:
training_samples = train.samples
training_steps = training_samples / 20
#setting up the batches
validation_samples = validate.samples
validation_steps = validation_samples / 20

In [120]:
#Finally, training the autoencoder! This takes about two hours on the cloud with a gpu.
history = autoencoder.fit(
      train,
      steps_per_epoch=training_steps,
      epochs=15,
      verbose = True,
      validation_data=validate,
      validation_steps=validation_steps,
      callbacks=[checkpoint])

Epoch 1/15
505/504 [==============================] - 730s 1s/step - loss: 0.0103 - acc: 0.7695 - val_loss: 0.0123 - val_acc: 0.7108

Epoch 00001: val_loss improved from inf to 0.01233, saving model to /home/stepianno/auto_saving.h5
Epoch 2/15
505/504 [==============================] - 729s 1s/step - loss: 0.0100 - acc: 0.7732 - val_loss: 0.0103 - val_acc: 0.8029

Epoch 00002: val_loss improved from 0.01233 to 0.01025, saving model to /home/stepianno/auto_saving.h5
Epoch 3/15
505/504 [==============================] - 727s 1s/step - loss: 0.0099 - acc: 0.7678 - val_loss: 0.0102 - val_acc: 0.8045

Epoch 00003: val_loss improved from 0.01025 to 0.01018, saving model to /home/stepianno/auto_saving.h5
Epoch 4/15
505/504 [==============================] - 728s 1s/step - loss: 0.0096 - acc: 0.7765 - val_loss: 0.0068 - val_acc: 0.8069

Epoch 00004: val_loss improved from 0.01018 to 0.00678, saving model to /home/stepianno/auto_saving.h5
Epoch 5/15
505/504 [==============================] - 72

In [130]:
autoencoder = load_model('../auto_saving.h5') #in order to load the saved autoencoder when returning

*In order to make new images with the autoencoder, I need to aplit it in half and save both the encoder and decoder separately. The encoder is a bit easier to save as it has the same input shape as the autoencoder. For the decoder, I must reconstruct the second half of the autoencoder and load the weights by name.*

In [153]:
feature_extractor = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer('encoded').output)
feature_extractor.save('../feature.h5')

In [ ]:
input_ = Input(shape=(28,28,16))

x = Conv2D(16, (3,3), activation = 'relu', padding='same', name='conv2d_11')(input_)
x = UpSampling2D((2,2), name='up_sampling2d_4')(x)
x = Conv2D(64, (3,3), activation = 'relu', padding='same', name='conv2d_12')(x)
x = UpSampling2D((2,2), name='up_sampling2d_5')(x)
x = Conv2D(512, (3,3), activation = 'relu', padding='same', name='conv2d_13')(x)
x = UpSampling2D((2,2), name='up_sampling2d_6')(x)
decoded = Conv2D(3, (3,3), activation = 'sigmoid', padding='same', name='conv2d_14')(x)

image_maker = Model(input_, decoded)
image_maker.load_weights('../auto_saving.h5', by_name=True)

In [ ]:
image_maker.save('/home/stepianno/imager.h5')